# Ejercicios Pair Programming - Modulo 2 - Sprint 2 
## Lección 2 - ETL II: Transformación I, Limpieza
### Ana Gonzalez y Sonia Ruiz

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:
1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado *(attacks_limpieza_completa)*. Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.
El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
- Cargaremos los dos ficheros de datos

In [29]:
#Importamos las librerias necesarias para poder trabajar en este archivo de jupyter.
import pandas as pd
import numpy as np
import requests

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all"

In [111]:
df_clima=pd.read_pickle('files/tiempo_paises2.pkl')

In [ ]:
#Cargamos los datos del dataframe tiempo_paises2 desde un pickle porque al hacerlo desde un .csv, no nos
#  deja separar las columnas 'rh_profile' y 'wind_profile' al aplicar los metodos:
    #Manera 1: df_clima['rh_profile'].apply(pd.Series)
    #Manera 2: pd.json_normalize(df_clima['rh_profile'])
#Pero al cargar el dataframe desde el pickle funciona bien.

In [65]:
df_ataques=pd.read_csv('files/attacks_limpieza_completa.csv',index_col=0)
df_ataques.head(3)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629


In [112]:
df_clima.head(3)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA
2,9,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1012,none,0,0,165,3,USA


- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [4]:
#Observamos el numero de valores unicos de la columna 'country'.
df_ataques['country'].value_counts()

usa                      2197
australia                1314
south africa              572
new zealand               126
papua new guinea          126
                         ... 
argentina                   1
sudan?                      1
the balkans                 1
north atlantic ocean        1
ceylon (sri lanka)          1
Name: country, Length: 210, dtype: int64

In [8]:
#Creamos un dataframe solo con los ataques que han ocurrido en los 5 paises que nos interesan.
df_ataques_paises=df_ataques[df_ataques['country'].isin(['usa','australia','south africa','new zealand','papua new guinea'])]
df_ataques_paises['country'].value_counts()

usa                 2197
australia           1314
south africa         572
new zealand          126
papua new guinea     126
Name: country, dtype: int64

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [113]:
#comprobamos el numero de filas y columas que tiene el dataframe df_clima.
df_clima.shape

(320, 17)

In [114]:
#Creamos un dataframe con la informacion contenida en la columna 'rh_profile', donde cada columna
#  corresponde a uno de los dirrcionarios que estaban almacenados en cada celda de cada fila.
df_rh=df_clima['rh_profile'].apply(pd.Series)
df_rh.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 2}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 8}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': 11}","{'layer': '200mb', 'rh': 3}"
1,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 2}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 12}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': 7}","{'layer': '350mb', 'rh': 10}","{'layer': '300mb', 'rh': 11}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"
2,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 5}","{'layer': '600mb', 'rh': 14}","{'layer': '550mb', 'rh': 6}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 11}","{'layer': '300mb', 'rh': 14}","{'layer': '250mb', 'rh': 14}","{'layer': '200mb', 'rh': 11}"


In [115]:
#Hacemos el mismo procedimiento con la columna 'wind_profile'.
df_wind=df_clima['wind_profile'].apply(pd.Series)
df_wind.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
1,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 195, 'speed': 3}","{'layer': '850mb', 'direction': 205, 'speed': 4}","{'layer': '800mb', 'direction': 220, 'speed': 4}","{'layer': '750mb', 'direction': 235, 'speed': 4}","{'layer': '700mb', 'direction': 250, 'speed': 5}","{'layer': '650mb', 'direction': 260, 'speed': 5}","{'layer': '600mb', 'direction': 300, 'speed': 6}","{'layer': '550mb', 'direction': 295, 'speed': 6}","{'layer': '500mb', 'direction': 285, 'speed': 6}","{'layer': '450mb', 'direction': 290, 'speed': 6}","{'layer': '400mb', 'direction': 295, 'speed': 6}","{'layer': '350mb', 'direction': 300, 'speed': 7}","{'layer': '300mb', 'direction': 300, 'speed': 9}","{'layer': '250mb', 'direction': 310, 'speed': 10}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
2,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 185, 'speed': 4}","{'layer': '850mb', 'direction': 200, 'speed': 5}","{'layer': '800mb', 'direction': 210, 'speed': 5}","{'layer': '750mb', 'direction': 225, 'speed': 5}","{'layer': '700mb', 'direction': 245, 'speed': 5}","{'layer': '650mb', 'direction': 250, 'speed': 5}","{'layer': '600mb', 'direction': 285, 'speed': 5}","{'layer': '550mb', 'direction': 280, 'speed': 6}","{'layer': '500mb', 'direction': 270, 'speed': 6}","{'layer': '450mb', 'direction': 275, 'speed': 6}","{'layer': '400mb', 'direction': 285, 'speed': 7}","{'layer': '350mb', 'direction': 280, 'speed': 7}","{'layer': '300mb', 'direction': 290, 'speed': 7}","{'layer': '250mb', 'direction': 295, 'speed': 7}","{'layer': '200mb', 'direction': 305, 'speed': 7}"


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas *rh_profile* y *wind_profile* cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consiguierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

````python
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.


# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 
```

In [116]:
#Separamos la columna rh_profile:
# Hacemos un for para iterar por cada una de las columnas. 
for i in range(len(df_rh.columns)): 

    # aplicamos el apply,extraemos el valores de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_rh[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

In [118]:
#Comprobamos las columnas que tenemos ahora en el dataframe df_clima.
df_clima.columns

Index(['rh_0    950mb\n0    950mb\n0    950mb\n0    950mb\n0    950mb\nName: layer, dtype: object',
       'rh_0    900mb\n0    900mb\n0    900mb\n0    900mb\n0    900mb\nName: layer, dtype: object',
       'rh_0    850mb\n0    850mb\n0    850mb\n0    850mb\n0    850mb\nName: layer, dtype: object',
       'rh_0    800mb\n0    800mb\n0    800mb\n0    800mb\n0    800mb\nName: layer, dtype: object',
       'rh_0    750mb\n0    750mb\n0    750mb\n0    750mb\n0    750mb\nName: layer, dtype: object',
       'rh_0    700mb\n0    700mb\n0    700mb\n0    700mb\n0    700mb\nName: layer, dtype: object',
       'rh_0    650mb\n0    650mb\n0    650mb\n0    650mb\n0    650mb\nName: layer, dtype: object',
       'rh_0    600mb\n0    600mb\n0    600mb\n0    600mb\n0    600mb\nName: layer, dtype: object',
       'rh_0    550mb\n0    550mb\n0    550mb\n0    550mb\n0    550mb\nName: layer, dtype: object',
       'rh_0    500mb\n0    500mb\n0    500mb\n0    500mb\n0    500mb\nName: layer, dtype: object',


In [123]:
#Comprobamos como ejemplo las filas pertenecientes solo a 'USA'.
df_clima[df_clima['country']=='USA']

,"rh_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","rh_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","rh_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","rh_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","rh_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","rh_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","rh_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","rh_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","rh_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object","rh_0 500mb\n0 500mb\n0 500mb\n0 500mb\n0 500mb\nName: layer, dtype: object",...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,3,2,2,3,5,4,2,11,14,...,4,15,10,1015,none,0,0,235,2,USA
1,-9999,2,2,2,1,3,4,12,7,3,...,4,15,8,1014,none,0,0,180,3,USA
2,-9999,3,0,0,1,3,5,14,6,3,...,3,15,7,1012,none,0,0,165,3,USA
3,-9999,2,0,0,1,3,6,9,7,7,...,2,15,7,1011,none,0,0,190,2,USA
4,-9999,3,1,1,2,4,7,8,6,6,...,3,15,6,1010,none,0,0,245,2,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,-9999,11,4,4,7,7,4,4,0,-2,...,2,15,13,1006,none,1,0,80,2,USA
60,-9999,12,7,7,9,9,5,10,11,16,...,2,10,12,1008,none,1,0,115,2,USA
61,-9999,12,10,12,12,13,14,15,16,16,...,3,6,10,1006,rain,1,0,95,3,USA
62,-9999,10,14,11,15,15,15,14,11,8,...,7,6,8,1005,rain,2,0,85,3,USA


In [124]:
#Comrobamos la informacion contenida en el dataframe df_wind.
df_wind.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
1,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 195, 'speed': 3}","{'layer': '850mb', 'direction': 205, 'speed': 4}","{'layer': '800mb', 'direction': 220, 'speed': 4}","{'layer': '750mb', 'direction': 235, 'speed': 4}","{'layer': '700mb', 'direction': 250, 'speed': 5}","{'layer': '650mb', 'direction': 260, 'speed': 5}","{'layer': '600mb', 'direction': 300, 'speed': 6}","{'layer': '550mb', 'direction': 295, 'speed': 6}","{'layer': '500mb', 'direction': 285, 'speed': 6}","{'layer': '450mb', 'direction': 290, 'speed': 6}","{'layer': '400mb', 'direction': 295, 'speed': 6}","{'layer': '350mb', 'direction': 300, 'speed': 7}","{'layer': '300mb', 'direction': 300, 'speed': 9}","{'layer': '250mb', 'direction': 310, 'speed': 10}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
2,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 185, 'speed': 4}","{'layer': '850mb', 'direction': 200, 'speed': 5}","{'layer': '800mb', 'direction': 210, 'speed': 5}","{'layer': '750mb', 'direction': 225, 'speed': 5}","{'layer': '700mb', 'direction': 245, 'speed': 5}","{'layer': '650mb', 'direction': 250, 'speed': 5}","{'layer': '600mb', 'direction': 285, 'speed': 5}","{'layer': '550mb', 'direction': 280, 'speed': 6}","{'layer': '500mb', 'direction': 270, 'speed': 6}","{'layer': '450mb', 'direction': 275, 'speed': 6}","{'layer': '400mb', 'direction': 285, 'speed': 7}","{'layer': '350mb', 'direction': 280, 'speed': 7}","{'layer': '300mb', 'direction': 290, 'speed': 7}","{'layer': '250mb', 'direction': 295, 'speed': 7}","{'layer': '200mb', 'direction': 305, 'speed': 7}"


In [125]:
#Separamos la columna wind_profile:
# Empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(df_wind.columns)): 

    # aplicamos el apply,extraemos el valores de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_dir_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
    nombre2 = "wind_spe_" + str(df_wind[i].apply(pd.Series)["layer"][0])

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    direccion = list(df_wind[i].apply(pd.Series)["direction"])

    velocidad= list(df_wind[i].apply(pd.Series)["speed"])

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, direccion)
    df_clima.insert(i, nombre2, velocidad)


In [131]:
#Hacemos un reset index del dataframe df_clima.
df_clima=df_clima.reset_index()

In [132]:
#Comprobamos las columnas que tenemos ahora en el dataframe df_clima.
df_clima.columns

Index(['index',
       'wind_spe_0    950mb\n0    950mb\n0    950mb\n0    950mb\n0    950mb\nName: layer, dtype: object',
       'wind_spe_0    900mb\n0    900mb\n0    900mb\n0    900mb\n0    900mb\nName: layer, dtype: object',
       'wind_spe_0    850mb\n0    850mb\n0    850mb\n0    850mb\n0    850mb\nName: layer, dtype: object',
       'wind_spe_0    800mb\n0    800mb\n0    800mb\n0    800mb\n0    800mb\nName: layer, dtype: object',
       'wind_spe_0    750mb\n0    750mb\n0    750mb\n0    750mb\n0    750mb\nName: layer, dtype: object',
       'wind_spe_0    700mb\n0    700mb\n0    700mb\n0    700mb\n0    700mb\nName: layer, dtype: object',
       'wind_spe_0    650mb\n0    650mb\n0    650mb\n0    650mb\n0    650mb\nName: layer, dtype: object',
       'wind_spe_0    600mb\n0    600mb\n0    600mb\n0    600mb\n0    600mb\nName: layer, dtype: object',
       'wind_spe_0    550mb\n0    550mb\n0    550mb\n0    550mb\n0    550mb\nName: layer, dtype: object',
       'wind_spe_0    500mb\n0

In [133]:
df_clima.head()

,index,"wind_spe_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","wind_spe_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","wind_spe_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","wind_spe_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","wind_spe_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","wind_spe_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","wind_spe_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","wind_spe_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","wind_spe_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object",...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,-9999,3,3,5,5,6,6,5,6,...,4,15,10,1015,none,0,0,235,2,USA
1,1,-9999,3,4,4,4,5,5,6,6,...,4,15,8,1014,none,0,0,180,3,USA
2,2,-9999,4,5,5,5,5,5,5,6,...,3,15,7,1012,none,0,0,165,3,USA
3,3,-9999,5,5,5,4,5,5,5,6,...,2,15,7,1011,none,0,0,190,2,USA
4,4,-9999,3,4,3,3,4,5,5,5,...,3,15,6,1010,none,0,0,245,2,USA


In [146]:
#Eliminamos las columnas de las que hemos sacado la informacion y ya no nos interesan.
df_clima2=df_clima.drop(['rh_profile','wind_profile'],inplace=False,axis=1)
df_clima2

,index,"wind_spe_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","wind_spe_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","wind_spe_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","wind_spe_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","wind_spe_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","wind_spe_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","wind_spe_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","wind_spe_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","wind_spe_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object",...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,-9999,3,3,5,5,6,6,5,6,...,4,15,10,1015,none,0,0,235,2,USA
1,1,-9999,3,4,4,4,5,5,6,6,...,4,15,8,1014,none,0,0,180,3,USA
2,2,-9999,4,5,5,5,5,5,5,6,...,3,15,7,1012,none,0,0,165,3,USA
3,3,-9999,5,5,5,4,5,5,5,6,...,2,15,7,1011,none,0,0,190,2,USA
4,4,-9999,3,4,3,3,4,5,5,5,...,3,15,6,1010,none,0,0,245,2,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,59,-9999,-9999,-9999,2,2,2,2,2,3,...,12,2,15,-9999,rain,6,0,250,2,Papua New Guinea
316,60,-9999,-9999,-9999,2,2,2,2,2,3,...,11,2,15,-9999,rain,6,0,270,2,Papua New Guinea
317,61,-9999,-9999,-9999,2,2,2,2,3,3,...,11,2,15,-9999,rain,6,0,280,2,Papua New Guinea
318,62,-9999,-9999,-9999,2,2,2,2,3,3,...,12,2,15,-9999,none,6,0,270,2,Papua New Guinea


In [150]:
#Comprobamos el numero de valores nulos que tenemos.
df_clima2.isnull().sum()

index                                                                                              0
wind_spe_0    950mb\n0    950mb\n0    950mb\n0    950mb\n0    950mb\nName: layer, dtype: object    0
wind_spe_0    900mb\n0    900mb\n0    900mb\n0    900mb\n0    900mb\nName: layer, dtype: object    0
wind_spe_0    850mb\n0    850mb\n0    850mb\n0    850mb\n0    850mb\nName: layer, dtype: object    0
wind_spe_0    800mb\n0    800mb\n0    800mb\n0    800mb\n0    800mb\nName: layer, dtype: object    0
                                                                                                  ..
prec_amount                                                                                        0
snow_depth                                                                                         0
wind10m.direction                                                                                  0
wind10m.speed                                                                              

In [157]:
#Hacemos el groupby por pais y sacamos la media de los valores para cada pais en cada columna.
df_agrupado=df_clima2.groupby('country').mean()
df_agrupado.reset_index()

/tmp/ipykernel_14329/4104068927.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agrupado=df_clima2.groupby('country').mean()


,country,index,"wind_spe_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","wind_spe_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","wind_spe_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","wind_spe_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","wind_spe_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","wind_spe_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","wind_spe_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","wind_spe_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object",...,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
0,Australia,31.5,3.12500,3.093750,3.312500,3.500000,3.687500,3.68750,3.390625,3.078125,...,-9999.0,-9999.0,-9999.0,32.625000,-1.140625,3.187500,1005.531250,2.640625,0.00000,2.734375
1,New Zealand,31.5,-2654.46875,2.203125,2.296875,2.515625,2.906250,2.81250,2.750000,2.828125,...,-9999.0,-9999.0,-9999.0,15.046875,5.078125,11.109375,1015.578125,2.921875,0.00000,1.953125
2,Papua New Guinea,31.5,-9999.00000,-9999.000000,-9999.000000,2.000000,2.000000,2.15625,2.390625,2.312500,...,-9999.0,-9999.0,-9999.0,14.906250,0.015625,13.218750,-9999.000000,4.546875,0.00000,1.984375
3,South Africa,31.5,-9999.00000,-9999.000000,3.531250,3.125000,2.718750,2.50000,2.609375,2.843750,...,-9999.0,-9999.0,-9999.0,31.640625,0.703125,0.140625,1010.062500,0.765625,0.00000,3.015625
4,USA,31.5,-9999.00000,3.234375,3.437500,3.578125,3.890625,4.12500,4.328125,4.578125,...,-9999.0,-9999.0,-9999.0,3.937500,13.218750,8.609375,1012.078125,0.796875,0.09375,2.421875


In [174]:
#Eliminamos la columna index y transformamos los valores de la columna country a minusculas, para poder hacer el 'mergeado' después 
# in problemas teniendo la misma tipologia en los dos Dataframes.
df_clima3=df_agrupado.drop(['index'],inplace=False,axis=1)
df_clima3.reset_index(inplace=True)
df_clima3['country']=df_clima3['country'].str.lower()
df_clima3

,country,"wind_spe_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","wind_spe_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","wind_spe_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","wind_spe_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","wind_spe_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","wind_spe_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","wind_spe_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","wind_spe_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","wind_spe_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object",...,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
0,australia,3.12500,3.093750,3.312500,3.500000,3.687500,3.68750,3.390625,3.078125,2.796875,...,-9999.0,-9999.0,-9999.0,32.625000,-1.140625,3.187500,1005.531250,2.640625,0.00000,2.734375
1,new zealand,-2654.46875,2.203125,2.296875,2.515625,2.906250,2.81250,2.750000,2.828125,2.968750,...,-9999.0,-9999.0,-9999.0,15.046875,5.078125,11.109375,1015.578125,2.921875,0.00000,1.953125
2,papua new guinea,-9999.00000,-9999.000000,-9999.000000,2.000000,2.000000,2.15625,2.390625,2.312500,2.718750,...,-9999.0,-9999.0,-9999.0,14.906250,0.015625,13.218750,-9999.000000,4.546875,0.00000,1.984375
3,south africa,-9999.00000,-9999.000000,3.531250,3.125000,2.718750,2.50000,2.609375,2.843750,2.921875,...,-9999.0,-9999.0,-9999.0,31.640625,0.703125,0.140625,1010.062500,0.765625,0.00000,3.015625
4,usa,-9999.00000,3.234375,3.437500,3.578125,3.890625,4.12500,4.328125,4.578125,4.875000,...,-9999.0,-9999.0,-9999.0,3.937500,13.218750,8.609375,1012.078125,0.796875,0.09375,2.421875


In [167]:
#Comprobamos las columnas que tenemos ahora en el dataframe df_clima3.
df_clima3.columns

Index(['country',
       'wind_spe_0    950mb\n0    950mb\n0    950mb\n0    950mb\n0    950mb\nName: layer, dtype: object',
       'wind_spe_0    900mb\n0    900mb\n0    900mb\n0    900mb\n0    900mb\nName: layer, dtype: object',
       'wind_spe_0    850mb\n0    850mb\n0    850mb\n0    850mb\n0    850mb\nName: layer, dtype: object',
       'wind_spe_0    800mb\n0    800mb\n0    800mb\n0    800mb\n0    800mb\nName: layer, dtype: object',
       'wind_spe_0    750mb\n0    750mb\n0    750mb\n0    750mb\n0    750mb\nName: layer, dtype: object',
       'wind_spe_0    700mb\n0    700mb\n0    700mb\n0    700mb\n0    700mb\nName: layer, dtype: object',
       'wind_spe_0    650mb\n0    650mb\n0    650mb\n0    650mb\n0    650mb\nName: layer, dtype: object',
       'wind_spe_0    600mb\n0    600mb\n0    600mb\n0    600mb\n0    600mb\nName: layer, dtype: object',
       'wind_spe_0    550mb\n0    550mb\n0    550mb\n0    550mb\n0    550mb\nName: layer, dtype: object',
       'wind_spe_0    500mb\

In [178]:
#Vemos que ahora todos los valores unicos de la columna country estan en minusculas y son solo los paises
#  que nos interesan.
df_clima3['country'].unique()

array(['australia', 'new zealand', 'papua new guinea', 'south africa',
       'usa'], dtype=object)

In [168]:
#Comprobamos las columnas que tenemos en el dataframe df_ataques_paises.
df_ataques_paises.columns

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud', 'country2', 'fatal_N', 'fatal_Unknown',
       'fatal_Y', 'fatal_N.1', 'fatal_Unknown.1', 'fatal_Y.1', 'species_.1',
       'fecha_limpia.1', 'type.1', 'age_NORM'],
      dtype='object')

In [177]:
#Comprobamos cuales son los valores unicos de la columna country.
df_ataques_paises['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [175]:
#Unimos ambos dataframes empleando la funcion merge haciendo un left join, y que nos interesa añadir
#  la informacion del clima al dataframe de los ataques, utilizando como columna de union 'country'.
df_mergeado=pd.merge(left=df_ataques_paises,right=df_clima3,how='left',left_on='country',right_on='country')
df_mergeado.head(3)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,-9999.0,-9999.0,-9999.0,3.9375,13.21875,8.609375,1012.078125,0.796875,0.09375,2.421875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,-9999.0,-9999.0,-9999.0,3.9375,13.21875,8.609375,1012.078125,0.796875,0.09375,2.421875
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,...,-9999.0,-9999.0,-9999.0,3.9375,13.21875,8.609375,1012.078125,0.796875,0.09375,2.421875


In [179]:
#Comprobamos el numero de valores nulos que tenemos en el nuevo dataframe.
df_mergeado.isnull().sum()

year             0
type             0
country          0
age              0
species_         0
                ..
rh2m             0
msl_pressure     0
prec_amount      0
snow_depth       0
wind10m.speed    0
Length: 81, dtype: int64

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [181]:
df_mergeado.to_pickle('files/tiempo_etl2.pkl')
df_mergeado.to_csv('files/tiempo_etl2.csv')

In [ ]:
#Guardamos el df_ataques_paises para poder utilizarlo en el pair IV de ETL.
df_ataques_paises.to_pickle('files/df_ataques_paises.pkl')

In [ ]:
#Guardamos el df_clima3 para poder utilizarlo en el pair IV de ETL.
df_clima3.to_pickle('files/df_clima3_paises.pkl')

**Happy coding** 🦈